In [2]:
import pandas as pd
import numpy as np

import plotly.express as px

In [13]:
data_path = '../data/hcvdat0.csv'
df = pd.read_csv(data_path, index_col=0)
df[['Category', 'Category_name']] = df.apply(lambda x: x.Category.split('='), axis=1, result_type='expand')
df.head()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,Category_name
1,0,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0,Blood Donor
2,0,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5,Blood Donor
3,0,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3,Blood Donor
4,0,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7,Blood Donor
5,0,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7,Blood Donor


In [16]:
numerical_columns = set(df.columns) - {'Category', 'Sex', 'Category_name'}
df_eda = df[list(numerical_columns)]
df_eda.head()

,CREA,CHOL,ALP,GGT,ALT,BIL,PROT,AST,CHE,ALB,Age
1,106.0,3.23,52.5,12.1,7.7,7.5,69.0,22.1,6.93,38.5,32
2,74.0,4.80,70.3,15.6,18.0,3.9,76.5,24.7,11.17,38.5,32
3,86.0,5.20,74.7,33.2,36.2,6.1,79.3,52.6,8.84,46.9,32
4,80.0,4.74,52.0,33.8,30.6,18.9,75.7,22.6,7.33,43.2,32
5,76.0,4.32,74.1,29.9,32.6,9.6,68.7,24.8,9.15,39.2,32
